This Hotel Reservations dataset contains real hotel demand data. It appeared in Nuno Antonio, Ana de Almeida, Luis Nunes. Hotel booking demand datasets. Data in Brief, 22 (2019): 41-49: https://www.sciencedirect.com/science/article/pii/S2352340918315191

Data for each instance (observation) is referred to as a hotel booking due to arrive between 2017 and 2018, including bookings that effectively arrived and bookings that were canceled.

The original raw dataset contains 36,275 instances. It is cleaned, preprocessed, and prepared in this notebook. After this phase of data preparation, a final dataset of 36,270 cases is obtained with sisteen attributes individually normalized with a min-max scaling, $\frac{x-min}{max-min}$  The file **HotelReservationsPreparedCleanAttributes.csv** contains the resulting dataset.

From this data, the classification problem consists of predicting whether the customer is going to honor the reservation or cancel it. Each class is labeled with 0 (canceled) or 1 (not canceled), stored in the file <b>HotelReservationsOutput.csv</b> for supervised training models.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, minmax_scale, scale
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
INPUT_FILE_NAME = "/gdrive/MyDrive/Máster/Redes de neuronas artificiales y DL/Entrega/HotelReservationsRawDataset.csv"
ATT_FILE_NAME = "/gdrive/MyDrive/Máster/Redes de neuronas artificiales y DL/Entrega/HotelReservationsPreparedCleanAttributes.csv"
OUTPUT_FILE_NAME = "/gdrive/MyDrive/Máster/Redes de neuronas artificiales y DL/Entrega/HotelReservationsOutput.csv"

In [ ]:
dataset = pd.read_csv(INPUT_FILE_NAME)

In [ ]:
type(dataset)

pandas.core.frame.DataFrame

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36275 entries, 0 to 36274
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Booking_ID                            36275 non-null  object 
 1   no_of_adults                          36275 non-null  int64  
 2   no_of_children                        36275 non-null  int64  
 3   no_of_weekend_nights                  36275 non-null  int64  
 4   no_of_week_nights                     36275 non-null  int64  
 5   type_of_meal_plan                     36275 non-null  object 
 6   required_car_parking_space            36275 non-null  int64  
 7   room_type_reserved                    36275 non-null  object 
 8   lead_time                             36275 non-null  int64  
 9   arrival_year                          36275 non-null  int64  
 10  arrival_month                         36275 non-null  int64  
 11  arrival_date   

In [ ]:
dataset[:10]

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled
5,INN00006,2,0,0,2,Meal Plan 2,0,Room_Type 1,346,2018,9,13,Online,0,0,0,115.00,1,Canceled
6,INN00007,2,0,1,3,Meal Plan 1,0,Room_Type 1,34,2017,10,15,Online,0,0,0,107.55,1,Not_Canceled
7,INN00008,2,0,1,3,Meal Plan 1,0,Room_Type 4,83,2018,12,26,Online,0,0,0,105.61,1,Not_Canceled
8,INN00009,3,0,0,4,Meal Plan 1,0,Room_Type 1,121,2018,7,6,Offline,0,0,0,96.90,1,Not_Canceled
9,INN00010,2,0,0,5,Meal Plan 1,0,Room_Type 4,44,2018,10,18,Online,0,0,0,133.44,3,Not_Canceled


In [ ]:
dataset[-10:]

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
36265,INN36266,2,0,1,3,Meal Plan 1,0,Room_Type 1,15,2018,5,30,Online,0,0,0,100.73,0,Not_Canceled
36266,INN36267,2,0,2,2,Meal Plan 1,0,Room_Type 2,8,2018,3,4,Online,0,0,0,85.96,1,Canceled
36267,INN36268,2,0,1,0,Not Selected,0,Room_Type 1,49,2018,7,11,Online,0,0,0,93.15,0,Canceled
36268,INN36269,1,0,0,3,Meal Plan 1,0,Room_Type 1,166,2018,11,1,Offline,0,0,0,110.00,0,Canceled
36269,INN36270,2,2,0,1,Meal Plan 1,0,Room_Type 6,0,2018,10,6,Online,0,0,0,216.00,0,Canceled
36270,INN36271,3,0,2,6,Meal Plan 1,0,Room_Type 4,85,2018,8,3,Online,0,0,0,167.80,1,Not_Canceled
36271,INN36272,2,0,1,3,Meal Plan 1,0,Room_Type 1,228,2018,10,17,Online,0,0,0,90.95,2,Canceled
36272,INN36273,2,0,2,6,Meal Plan 1,0,Room_Type 1,148,2018,7,1,Online,0,0,0,98.39,2,Not_Canceled
36273,INN36274,2,0,0,3,Not Selected,0,Room_Type 1,63,2018,4,21,Online,0,0,0,94.50,0,Canceled
36274,INN36275,2,0,1,2,Meal Plan 1,0,Room_Type 1,207,2018,12,30,Offline,0,0,0,161.67,0,Not_Canceled


**First Step:** find out whether or not there are missing values and, in such case, remove them.

In [ ]:
{att : dataset[dataset[att].isnull()].shape[0] for att in dataset.columns}

{'Booking_ID': 0,
 'no_of_adults': 0,
 'no_of_children': 0,
 'no_of_weekend_nights': 0,
 'no_of_week_nights': 0,
 'type_of_meal_plan': 0,
 'required_car_parking_space': 0,
 'room_type_reserved': 0,
 'lead_time': 0,
 'arrival_year': 0,
 'arrival_month': 0,
 'arrival_date': 0,
 'market_segment_type': 0,
 'repeated_guest': 0,
 'no_of_previous_cancellations': 0,
 'no_of_previous_bookings_not_canceled': 0,
 'avg_price_per_room': 0,
 'no_of_special_requests': 0,
 'booking_status': 0}

There are no missing values.

**Second Step:** For all non-numerical attributes check how many instances per labels there are.

In [ ]:
dataset.type_of_meal_plan.value_counts()

Meal Plan 1     27835
Not Selected     5130
Meal Plan 2      3305
Meal Plan 3         5
Name: type_of_meal_plan, dtype: int64

Meal Plan 3 is removed since there are only the following five examples.

In [ ]:
dataset.drop(dataset[dataset.type_of_meal_plan=="Meal Plan 3"].index,inplace=True)

In [ ]:
dataset.type_of_meal_plan.value_counts()

Meal Plan 1     27835
Not Selected     5130
Meal Plan 2      3305
Name: type_of_meal_plan, dtype: int64

In [ ]:
dataset.room_type_reserved.value_counts()

Room_Type 1    28129
Room_Type 4     6056
Room_Type 6      966
Room_Type 2      692
Room_Type 5      265
Room_Type 7      155
Room_Type 3        7
Name: room_type_reserved, dtype: int64

Grouping Room types 6, 2, 5, 7, and 3 as Other to balance the values.

In [ ]:
dataset['room_type_reserved'] = dataset['room_type_reserved'].replace(['Room_Type 6', 'Room_Type 2',
                                                  'Room_Type 5', 'Room_Type 7', 'Room_Type 3'], 'Other')
dataset.room_type_reserved.value_counts()

Room_Type 1    28129
Room_Type 4     6056
Other           2085
Name: room_type_reserved, dtype: int64

In [ ]:
dataset.market_segment_type.value_counts()

Online           23214
Offline          10527
Corporate         2017
Complementary      387
Aviation           125
Name: market_segment_type, dtype: int64

Grouping market segment types Corporate, Complementary, And Aviation as Other to balance the values.

In [ ]:
dataset['market_segment_type'] = dataset['market_segment_type'].replace(['Corporate', 'Complementary',
                                                  'Aviation'], 'Other')
dataset.market_segment_type.value_counts()

Online     23214
Offline    10527
Other       2529
Name: market_segment_type, dtype: int64

Removing Booking_ID and arrival_year attributes since they are not relevant. In the case of arrival_year, there are  only two values: 2017 and 2018.

In [ ]:
dataset = dataset.drop('arrival_year', axis=1)
dataset = dataset.drop('Booking_ID', axis=1)
dataset[:10]

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,2,0,2,3,Not Selected,0,Room_Type 1,5,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2,28,Online,0,0,0,60.00,0,Canceled
3,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,5,20,Online,0,0,0,100.00,0,Canceled
4,2,0,1,1,Not Selected,0,Room_Type 1,48,4,11,Online,0,0,0,94.50,0,Canceled
5,2,0,0,2,Meal Plan 2,0,Room_Type 1,346,9,13,Online,0,0,0,115.00,1,Canceled
6,2,0,1,3,Meal Plan 1,0,Room_Type 1,34,10,15,Online,0,0,0,107.55,1,Not_Canceled
7,2,0,1,3,Meal Plan 1,0,Room_Type 4,83,12,26,Online,0,0,0,105.61,1,Not_Canceled
8,3,0,0,4,Meal Plan 1,0,Room_Type 1,121,7,6,Offline,0,0,0,96.90,1,Not_Canceled
9,2,0,0,5,Meal Plan 1,0,Room_Type 4,44,10,18,Online,0,0,0,133.44,3,Not_Canceled


**Third Step:** shuffle the dataset (three times).

In [32]:
dataset=dataset.sample(frac=1) #frac is the fraction of axis items to return. 1 means all of them
dataset=dataset.sample(frac=1)
dataset=dataset.sample(frac=1).reset_index(drop=True) #Reset index and drop the old one
dataset.head()

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,2,0,1,3,Meal Plan 1,0,Room_Type 1,0,8,10,Online,0,0,0,124.00,0,Not_Canceled
1,2,0,2,3,Meal Plan 1,0,Room_Type 1,294,12,29,Online,0,0,0,82.11,1,Not_Canceled
2,1,0,2,1,Meal Plan 2,0,Room_Type 1,117,8,1,Offline,0,0,0,72.50,0,Not_Canceled
3,1,0,0,1,Meal Plan 1,0,Room_Type 1,3,11,24,Other,1,1,3,67.00,1,Not_Canceled
4,2,0,0,3,Meal Plan 1,0,Room_Type 1,84,8,6,Online,0,0,0,76.50,1,Canceled


**Fourth Step:** Encoding discrete attributes so that the neural network can compute their values.

In [33]:
encoder_meal = LabelEncoder() # Function that transform non-numeral labels into integers.
encoder_room = LabelEncoder()
encoder_market = LabelEncoder()
dataset.type_of_meal_plan = encoder_meal.fit_transform(dataset.type_of_meal_plan.values)
dataset.room_type_reserved = encoder_room.fit_transform(dataset.room_type_reserved.values)
dataset.market_segment_type = encoder_market.fit_transform(dataset.market_segment_type.values)

In [34]:
encoder_status = LabelEncoder()
dataset.booking_status = encoder_status.fit_transform(dataset.booking_status.values)
dataset.head()

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,2,0,1,3,0,0,1,0,8,10,1,0,0,0,124.00,0,1
1,2,0,2,3,0,0,1,294,12,29,1,0,0,0,82.11,1,1
2,1,0,2,1,1,0,1,117,8,1,0,0,0,0,72.50,0,1
3,1,0,0,1,0,0,1,3,11,24,2,1,1,3,67.00,1,1
4,2,0,0,3,0,0,1,84,8,6,1,0,0,0,76.50,1,0


In [35]:
{ value: encoder_status.inverse_transform([value]) for value in range(2)}

{0: array(['Canceled'], dtype=object),
 1: array(['Not_Canceled'], dtype=object)}

The **correlation matrix** permits to visualize dependencies between pairs of attributes: values close to -1 or +1 indicate a high correlation. A negative correlation rate means than when the value of an attribute gets high, the value of the other attribute decreases, and vice-versa. Positive correlation values suggest that both features increase or deacrease simultaneously. All attributes are fairly uncorrelated. It is noteworthy that the lead_time has the highest negative correlation (-0.43) with the booking status, the variable to predict. The number of adults and children are also quite high correlated with the average price per room, which seems reasonable.   

In [36]:
dataset.corr()

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
no_of_adults,1.000000,-0.019777,0.103350,0.105644,0.027156,0.011394,0.243383,0.097329,0.021882,0.026328,-0.074028,-0.192425,-0.047423,-0.119174,0.297138,0.189408,-0.086937
no_of_children,-0.019777,1.000000,0.029454,0.024384,-0.087733,0.034294,-0.354446,-0.047126,-0.003110,0.025492,0.084566,-0.036328,-0.016392,-0.021184,0.337867,0.124489,-0.033070
no_of_weekend_nights,0.103350,0.029454,1.000000,0.179564,-0.032794,-0.031178,0.042270,0.046516,-0.009947,0.027362,-0.003982,-0.066999,-0.020698,-0.026293,-0.004662,0.060579,-0.061556
no_of_week_nights,0.105644,0.024384,0.179564,1.000000,-0.086598,-0.048681,0.078529,0.149616,0.037320,-0.009301,-0.052123,-0.099681,-0.030085,-0.049325,0.022731,0.045990,-0.093013
type_of_meal_plan,0.027156,-0.087733,-0.032794,-0.086598,1.000000,-0.017046,-0.112649,-0.027767,0.012527,0.007870,0.023177,-0.067585,-0.013822,-0.040734,-0.052709,0.009087,-0.037529
required_car_parking_space,0.011394,0.034294,-0.031178,-0.048681,-0.017046,1.000000,-0.004472,-0.066348,-0.015203,0.000090,0.123701,0.111068,0.027126,0.063856,0.061793,0.087885,0.086123
room_type_reserved,0.243383,-0.354446,0.042270,0.078529,-0.112649,-0.004472,1.000000,-0.048931,-0.008629,0.008647,0.050561,-0.033588,-0.011161,-0.023256,0.091249,0.072829,-0.003044
lead_time,0.097329,-0.047126,0.046516,0.149616,-0.027767,-0.066348,-0.048931,1.000000,0.136711,0.006505,-0.323397,-0.135910,-0.045733,-0.078122,-0.062902,-0.101648,-0.438536
arrival_month,0.021882,-0.003110,-0.009947,0.037320,0.012527,-0.015203,-0.008629,0.136711,1.000000,-0.042817,-0.034053,0.000515,-0.038627,-0.010695,0.054005,0.110596,0.011354
arrival_date,0.026328,0.025492,0.027362,-0.009301,0.007870,0.000090,0.008647,0.006505,-0.042817,1.000000,0.011636,-0.016040,-0.012540,-0.001511,0.017874,0.018368,-0.010604


**Fifth Step:** The target output t is extracted

In [37]:
t = pd.DataFrame(data=dataset["booking_status"].values,columns=["booking_status"])
t[-10:]

,booking_status
36260,1
36261,0
36262,0
36263,1
36264,1
36265,1
36266,0
36267,1
36268,1
36269,1


In [38]:
dataset.drop(columns="booking_status", inplace=True)
dataset[:10]

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests
0,2,0,1,3,0,0,1,0,8,10,1,0,0,0,124.00,0
1,2,0,2,3,0,0,1,294,12,29,1,0,0,0,82.11,1
2,1,0,2,1,1,0,1,117,8,1,0,0,0,0,72.50,0
3,1,0,0,1,0,0,1,3,11,24,2,1,1,3,67.00,1
4,2,0,0,3,0,0,1,84,8,6,1,0,0,0,76.50,1
5,1,0,0,2,0,0,1,188,6,15,0,0,0,0,130.00,0
6,1,0,2,2,0,0,1,115,2,27,0,0,0,0,76.00,0
7,1,0,1,0,0,0,1,4,10,19,1,0,0,0,90.00,0
8,1,0,0,2,0,0,1,103,4,19,0,0,0,0,115.00,0
9,2,0,0,1,0,0,1,38,11,4,1,0,0,0,108.00,1


In [39]:
dataset.shape

(36270, 16)

**Sixth Step:** Standardization or Normalization of the input dataset. *Standardization* is much less affected by outliers, but it does not bound values to a specific range, which may be a problem for neural networks. The center is the mean, and the data is scaled to unit variance. On the other hand, *max-min scaling* ranges the data within max and min values, usually -1, and 1.  

In [40]:
x = pd.DataFrame (minmax_scale (dataset, feature_range=(-1,1)),columns=dataset.columns)
#x = pd.DataFrame (scale (x),columns=x.columns)

In [41]:
x[:10]

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests
0,0.0,-1.0,-0.714286,-0.647059,-1.0,-1.0,0.0,-1.000000,0.272727,-0.400000,0.0,-1.0,-1.000000,-1.000000,-0.540741,-1.0
1,0.0,-1.0,-0.428571,-0.647059,-1.0,-1.0,0.0,0.327314,1.000000,0.866667,0.0,-1.0,-1.000000,-1.000000,-0.695889,-0.6
2,-0.5,-1.0,-0.428571,-0.882353,0.0,-1.0,0.0,-0.471783,0.272727,-1.000000,-1.0,-1.0,-1.000000,-1.000000,-0.731481,-1.0
3,-0.5,-1.0,-1.000000,-0.882353,-1.0,-1.0,0.0,-0.986456,0.818182,0.533333,1.0,1.0,-0.846154,-0.896552,-0.751852,-0.6
4,0.0,-1.0,-1.000000,-0.647059,-1.0,-1.0,0.0,-0.620767,0.272727,-0.666667,0.0,-1.0,-1.000000,-1.000000,-0.716667,-0.6
5,-0.5,-1.0,-1.000000,-0.764706,-1.0,-1.0,0.0,-0.151242,-0.090909,-0.066667,-1.0,-1.0,-1.000000,-1.000000,-0.518519,-1.0
6,-0.5,-1.0,-0.428571,-0.764706,-1.0,-1.0,0.0,-0.480813,-0.818182,0.733333,-1.0,-1.0,-1.000000,-1.000000,-0.718519,-1.0
7,-0.5,-1.0,-0.714286,-1.000000,-1.0,-1.0,0.0,-0.981941,0.636364,0.200000,0.0,-1.0,-1.000000,-1.000000,-0.666667,-1.0
8,-0.5,-1.0,-1.000000,-0.764706,-1.0,-1.0,0.0,-0.534989,-0.454545,0.200000,-1.0,-1.0,-1.000000,-1.000000,-0.574074,-1.0
9,0.0,-1.0,-1.000000,-0.882353,-1.0,-1.0,0.0,-0.828442,0.818182,-0.800000,0.0,-1.0,-1.000000,-1.000000,-0.600000,-0.6


Some **descriptive statistics** on the attributes to confirm the mean and the variance.

In [42]:
x.describe().round(2)

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests
count,36270.00,36270.00,36270.00,36270.00,36270.00,36270.00,36270.00,36270.00,36270.00,36270.00,36270.00,36270.00,36270.00,36270.00,36270.00,36270.00
mean,-0.08,-0.98,-0.77,-0.74,-0.63,-0.94,0.11,-0.62,0.17,-0.03,-0.22,-0.95,-1.00,-0.99,-0.62,-0.75
std,0.26,0.08,0.25,0.17,0.72,0.35,0.46,0.39,0.56,0.58,0.56,0.32,0.06,0.06,0.13,0.31
min,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00
25%,0.00,-1.00,-1.00,-0.88,-1.00,-1.00,0.00,-0.92,-0.27,-0.53,-1.00,-1.00,-1.00,-1.00,-0.70,-1.00
50%,0.00,-1.00,-0.71,-0.76,-1.00,-1.00,0.00,-0.74,0.27,0.00,0.00,-1.00,-1.00,-1.00,-0.63,-1.00
75%,0.00,-1.00,-0.43,-0.65,-1.00,-1.00,0.00,-0.43,0.64,0.47,0.00,-1.00,-1.00,-1.00,-0.56,-0.60
max,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


**Saving** attributes matrix *x* target labels *t* and continuous target values to csv files.

In [43]:
x.to_csv (ATT_FILE_NAME, index = False)
t.to_csv (OUTPUT_FILE_NAME, index = False)